In [2]:
import pandas as pd
import numpy as np

import spacy
!python3 -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.0 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [3]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.3/en_ner_bc5cdr_md-0.5.3.tar.gz
ner = spacy.load("en_ner_bc5cdr_md")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 11.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 60.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 89.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 15.4 MB/s eta 0:00:00
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.5.3-py3-none-any.whl size=119790315 sha256=07930429495871732bcfde8be6a52142f13637174c9c0676e23741e5f1a41120
  Stored in directory: /root/.cache/pip/wheels/14/f5/28/d5bb4d966a84523c9135ec2113fb756fcc3098929f841334c1
Successfully built en-ner-bc5cdr-md
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.1
    Uninstalling thinc-8.2.1:
      Successfully uninstalled thinc-8.2.1
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
ERRO

/opt/conda/lib/python3.10/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_ner_bc5cdr_md' (0.5.3) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/opt/conda/lib/python3.10/site-packages/spacy/language.py:2170: FutureWarning: Possible set union at position 6328
  serializers["tokenizer"] = lambda: self.tokenizer.to_bytes(exclude=["vocab"])  # type: ignore[union-attr]


In [1]:
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 71.8 MB/s eta 0:00:00:00:0100:01


In [1]:
import requests
import time

def search_pmc_for_bcrp(api_key):
    base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
    db = "pmc"
    search_terms = "BCRP OR ABCG2"
    records_per_page = 10
    retries = 5
    retry_delay = 60  # Delay time in seconds

    # Perform the initial search to get the number of records
    search_url = f"{base_url}?db={db}&term={search_terms}&retmode=json&api_key={api_key}"
    response = requests.get(search_url)

    if response.status_code != 200:
        print(f"Failed to fetch data, Status Code: {response.status_code}")
        print(response.text)
        return []

    data = response.json()
    total_records = int(data.get("esearchresult", {}).get("count", 0))
    if total_records == 0:
        print("No records found.")
        return []

    article_list = []

    for start_record in range(0, total_records, records_per_page):
        for attempt in range(retries):
            search_url = f"{base_url}?db={db}&term={search_terms}&retmode=json&retstart={start_record}&retmax={records_per_page}&api_key={api_key}"
            response = requests.get(search_url)

            if response.status_code == 200:
                data = response.json()
                article_list.extend(data.get("esearchresult", {}).get("idlist", []))
                break  # Successful request, no need to retry
            elif response.status_code == 429:
                print(f"Rate limit hit. Retrying after {retry_delay} seconds...")
                time.sleep(retry_delay)
            else:
                print(f"Failed to fetch data for records starting at {start_record}, Status Code: {response.status_code}")
                print(response.text)
                break  # Break if there's an error other than rate limiting

    return article_list

if __name__ == "__main__":
    api_key = '1ff57034a28dd7b67afb3cbe1f921cc15809'  # Your API key
    art_list = search_pmc_for_bcrp(api_key)

In [6]:
for i in range(len(art_list)):
    art_list[i] = 'PMC'+art_list[i]
len(art_list)

24648

In [44]:
notparsed = []

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

import requests
import xml.etree.ElementTree as ET
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
!python3 -m spacy download en_core_web_lg
nlp = spacy.load("en_core_web_lg")


def download_xml_and_extract_text(pmcid):
    # Construct the URL for BioC XML
    bioC_url = f'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/{pmcid}/unicode'

    try:
        # Download XML content
        response = requests.get(bioC_url)
        response.raise_for_status()

        # Parse XML content
        root = etree.fromstring(xml_content_bytes, parser=parser)

        # Find all 'infon' elements with the attribute 'key' equal to 'article-id_pmid'
        pmid_elements = root.xpath('//infon[@key="article-id_pmid"]')

        # Extracting the text content of the found elements
        pmid = pmid.text for pmid in pmid_elements
        
        # Initialize variables for formatting
        formatted_text = ""
        current_section_type = None

        # Extract text content and format based on section types
        for passage in root.findall('.//passage'):
            section_type = passage.find("infon[@key='section_type']")
            if section_type is not None and section_type.text not in ['TABLE', 'REF', 'FIG', 'ABBR', 'SUPPL'] and passage.find("text") is not None: #
                section_text = passage.find("text").text.strip()

                # Add newline if a new section starts
                if current_section_type != section_type.text:
                    formatted_text += '\n'

                # Format based on section types
                formatted_text += f"\n## {section_type.text}\n{section_text}\n"

                current_section_type = section_type.text

        # Print or process the formatted text
        return formatted_text

    except requests.exceptions.HTTPError as err:
        print(f'Failed to download XML for PMC{pmcid}. HTTP Error: {err}')
        return None
    except ET.ParseError as parse_err:
        print(f'Error parsing XML: {parse_err}')
        return None


pmcid_to_download = art_list  
    
def clean_text(text):
    # Remove newline characters
    cleaned_text = text.replace('\n', ' ')
    cleaned_text = cleaned_text.replace('\t', ' ')

    # Process the cleaned text using spaCy
    #doc = nlp(cleaned_text)

    # Remove stop words
    #cleaned_tokens = [token.text for token in doc if token.text.lower() not in STOP_WORDS]

    # Join tokens back into text
    #cleaned_text = ' '.join(t.text for t in doc)

    return cleaned_text

In [97]:
import requests
import pandas as pd
from lxml import etree
import xml.etree.ElementTree as ET

def extract_table_from_xml(xml_string):
    """
    Extract the table into a pandas DataFrame from an XML string.
    """
    try:
        if xml_string is not None:
            # Parse the XML data, ignoring the XML declaration
            root = etree.fromstring(xml_string.encode('utf-8'))

            # Extract table headers using XPath
            header_cells = root.xpath('.//thead/tr/th')
            columns = ['Type' if not ''.join(cell.itertext()).strip() else ''.join(cell.itertext()).strip() for cell in header_cells]

            # Extract table rows using XPath
            body_rows = root.xpath('.//tbody/tr')
            data = [[cell.text.strip() if cell.text else ' ' for cell in row.xpath('.//td')] for row in body_rows]

            # Creating a DataFrame from the extracted data
            return pd.DataFrame(data)
        else:
            return None
    except:
        return None

def fetch_and_parse_paper(pmcid):
    """
    Request a paper by PMID in JSON format and parse it to extract important text and tables.
    """
    bioC_url = f'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/{pmcid}/unicode'

    try:
        # Download XML content
        response = requests.get(bioC_url)
        response.raise_for_status()

        # Parse XML content
        root = etree.fromstring(response.content)

        # Find all 'infon' elements with the attribute 'key' equal to 'article-id_pmid'
        pmid_elements = root.xpath('//infon[@key="article-id_pmid"]')

        # Extracting the text content of the found elements
        pmid = [pmid.text for pmid in pmid_elements]
        
        if len(pmid)>0:
            pmid = pmid[0]
            # Formulate the request URL
            url = f'https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_json/{pmid}/unicode'

            # Make the GET request
            response = requests.get(url)

            # If the response is successful, no Exception is raised
            response.raise_for_status()

            # Load the JSON content
            paper_json = response.json()

            # Initialize a list to hold all the important text content
            important_text_content = []

            # Initialize a list to store DataFrames for all tables found
            tables_dataframes = []

            # Loop through the JSON structure and extract text and tables
            for document in paper_json['documents']:
                for passage in document['passages']:
                    section_type = passage.get('infons', {}).get('section_type')
                    # Skip the undesired sections
                    if section_type not in ['REF', 'ABBR', 'FIG', 'SUPPL']:
                        # If it's a table, we will handle it separately
                        if section_type == 'TABLE' and passage['infons']['type'] == 'table':
                            # Extract and parse the XML from the table
                            table_xml = passage['infons']['xml']
                            # Convert XML to DataFrame using the function extract_table_from_xml
                            df_table = extract_table_from_xml(table_xml)
                            tables_dataframes.append(df_table)
                        else:
                            # Add the text content to the important text content list
                            important_text_content.append(passage['text'])

            return important_text_content, tables_dataframes
        else:
            print("Couln't parse JSON")
            notparsed.append(pmcid)
            return [None,None]
            
    
    except requests.exceptions.HTTPError as err:
        print(f'Failed to download XML for PMC{pmcid}. HTTP Error: {err}')
        return [None, None]
    except ET.ParseError as parse_err:
        print(f'Error parsing XML: {parse_err}')
        return [None, None]

# This function can now be called with a PMID to fetch and parse the paper.
# Example usage:
# important_text, table_dfs = fetch_and_parse_paper('21341745')


fetch_and_parse_paper(21341745)

(['A Novel Approach for Predicting P-glycoprotein (ABCB1) Inhibition Using Molecular Interaction Fields',
  "P-glycoprotein (Pgp or ABCB1) is an ABC transporter protein involved in intestinal absorption, drug metabolism and brain penetration, and its inhibition can seriously alter a drug's bioavailability and safety. In addition, inhibitors of Pgp can be used to overcome multidrug resistance. Given this dual-purpose, reliable in silico procedures to predict Pgp inhibition are of great interest. A large and accurate literature collection yielded more than 1200 structures; a model was then constructed using various MIF-based technologies, considering pharmacophoric features and those physico-chemical properties related to membrane partitioning. High accuracy was demonstrated internally, with two different validation sets, and moreover using a number of molecules, for which Pgp inhibition was not experimentally available but was evaluated `in-house'. All the validations confirmed the robu

In [19]:
art_list

['PMC10653841',
 'PMC10653122',
 'PMC10652209',
 'PMC10651738',
 'PMC10650504',
 'PMC10650073',
 'PMC10648718',
 'PMC10648204',
 'PMC10648040',
 'PMC10647199',
 'PMC10645471',
 'PMC10644303',
 'PMC10643871',
 'PMC10641871',
 'PMC10641764',
 'PMC10640796',
 'PMC10640763',
 'PMC10639169',
 'PMC10638691',
 'PMC10636723',
 'PMC10636675',
 'PMC10635512',
 'PMC10634604',
 'PMC10632904',
 'PMC10630959',
 'PMC10629827',
 'PMC10629271',
 'PMC10628734',
 'PMC10626554',
 'PMC10626523',
 'PMC10625943',
 'PMC10625179',
 'PMC10624793',
 'PMC10624720',
 'PMC10622746',
 'PMC10622546',
 'PMC10621072',
 'PMC10620457',
 'PMC10620275',
 'PMC10618919',
 'PMC10617216',
 'PMC10616432',
 'PMC10613986',
 'PMC10613910',
 'PMC10612515',
 'PMC10610652',
 'PMC10610388',
 'PMC10610386',
 'PMC10610319',
 'PMC10610137',
 'PMC10609930',
 'PMC10609764',
 'PMC10609725',
 'PMC10609717',
 'PMC10609273',
 'PMC10608541',
 'PMC10608504',
 'PMC10608342',
 'PMC10608301',
 'PMC10607849',
 'PMC10606708',
 'PMC10606328',
 'PMC106

In [49]:
import gc
from drug_named_entity_recognition import find_drugs

nlp.max_length = 200000000
def find_entities(PMCID):
    text, tables = fetch_and_parse_paper(PMCID)
    entity = []
    if len(text) != 0:

        bcpr_keywords = ["bcrp", "breast cancer resistance protein", "abcg2"]

        for i in text:
            doc = nlp(i)
            has_bcrp = any(token.text.lower() in bcpr_keywords for token in doc)
            if has_bcrp:
                ents = ner(i).ents
                drugs = []
                for e in ents:
                    if e.label_ == "CHEMICAL":
                        drugs.append(e.text)
                for drug in drugs:
                    entity.append({"drug": drug, 'context': doc.text, "Synonyms": None})
    
                entities = find_drugs([t.text for t in doc], is_ignore_case=True)
                if len(entities) > 0:
                    for e in entities :
                        entity.append({"drug": e[0]['name'], "context": doc.text, "Synonyms": e[0]['synonyms']})
                
        if len(entity) > 0:
            return entity,tables

find_entities(art_list[1])

([{'drug': 'tyrosine',
   'context': 'AlphaSpace has also guided the analysis of protein structures for targetable binding sites to design small molecule inhibitors and understand their binding mechanisms. Some computational experiments qualitatively studied the interaction mechanisms and binding modes by docking inhibitors to the most targetable pockets of the transmembrane protein ABCG2 and by running molecular dynamics (MD) simulations of the complexes. Other binding pocket analyses led to successful prospective docking studies. Hou et al. detected a cryptic pocket in a metastable state of the apo striatal-enriched protein tyrosine phosphatase (STEP) from MD simulations. Ensemble docking to this state at the cryptic pocket resulted in 11 novel inhibitors (IC50 = 9.7–47.7 μm). These experimental validations highlight the utility and role that AlphaSpace can play in identifying pockets to develop ligands with improved binding.',
   'Synonyms': None},
  {'drug': 'Chloroquine',
   'cont

In [98]:
import gc
from drug_named_entity_recognition import find_drugs

nlp.max_length = 200000000
def find_entities(PMCID):
    text, tables = fetch_and_parse_paper(PMCID)
    entity = []
    if text is not None:

        bcpr_keywords = ["bcrp", "breast cancer resistance protein", "abcg2"]

        for i in text:
            doc = nlp(i)
            has_bcrp = any(token.text.lower() in bcpr_keywords for token in doc)
            if has_bcrp:
                ents = ner(i).ents
                drugs = []
                for e in ents:
                    if e.label_ == "CHEMICAL":
                        drugs.append(e.text)
                for drug in drugs:
                    entity.append({"drug": drug, 'context': doc.text, "Synonyms": None, "PMCID": PMCID})
    
                entities = find_drugs([t.text for t in doc], is_ignore_case=True)
                if len(entities) > 0:
                    for e in entities :
                        entity.append({"drug": e[0]['name'], "context": doc.text, "Synonyms": e[0]['synonyms'], "PMCID": PMCID})
                
        return [entity,tables]
    else:
        return [None,None]
    
def process_in_batches(pmcid_list, batch_size=100):
    batch_results = []
    batch_results_tables = []
    x=0
    for index, PMCID in enumerate(pmcid_list):
        entities,tables = find_entities(PMCID)
        x+=1
        print(x)
        batch_results.append(entities)
        batch_results_tables.append(tables)
        if (index + 1) % batch_size == 0:
            yield [batch_results, batch_results_tables]  # Yield results as a batch
            batch_results.clear()  # Clear the list to free memory
            gc.collect()  # Collect garbage to free memory
    yield [batch_results, batch_results_tables]  # Yield any remaining results

batch_size = 200  # Define your batch size

extracted_drugs = []
# x = 0
# for PMCID in pmcid_to_download[0:500]:
#     entities = find_entities(PMCID)
#     extracted_drugs.append(entities)
#     x+= 1
#     if x%50 == 0:
#         print(extracted_drugs)

# Open a file to write the results
tables_all = []
with open('Final_Run.txt', 'w') as output_file:
    for b in process_in_batches(art_list, batch_size):
        batch = b[0]
        tables = b[1]
        tables_all.append(tables)
        for entities in batch:
            # Write entities to file or process them
            output_file.write(str(entities) + '\n')
        print(f"Processed a batch of {batch_size}")



1
2
Couln't parse JSON
3
Couln't parse JSON
4
5
6
7
8
9
Couln't parse JSON
10
11
Couln't parse JSON
12
Couln't parse JSON
13
14
15
Couln't parse JSON
16
17
18
19
20
Failed to download XML for PMCPMC10636675. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10636675/unicode
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
Failed to download XML for PMCPMC10560943. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10560943/unicode
122
Failed to download XML for PMCPMC10560931. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/R

Failed to download XML for PMCPMC10408485. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10408485/unicode
873
874
875
876
877
878
879
Failed to download XML for PMCPMC10396888. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10396888/unicode
880
Failed to download XML for PMCPMC10396865. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10396865/unicode
881
882
883
884
885
886
887
888
889
890
891
892
893
894
895
896
897
898
899
900
901
902
903
904
905
906
907
908
909
910
911
912
913
914
915
916
917
918
919
920
921
922
923
924
925
926
927
928
929
930
931
932
933
934
935
936
937
938
939
940
941
942
943
944
945
946
947
948
949
950
951
952
953
954
955
956
957
958
959
960
961
962
963
964
965
966
967
968
969
970
971
972
973
974
975
976
977
978
979
Couln't parse JSON
9

1631
1632
1633
Failed to download XML for PMCPMC10121400. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10121400/unicode
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
Failed to download XML for PMCPMC10074886. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10074886/unicode
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
Failed to download XML for PMCPMC10086885. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC10086885/unicode
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
Failed to download XML for PMCPMC10077047. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml

2166
2167
2168
2169
2170
2171
2172
2173
Failed to download XML for PMCPMC9662806. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC9662806/unicode
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
Processed a batch of 200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212
2213
2214
2215
2216
2217
2218
2219
Failed to download XML for PMCPMC9634445. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC9634445/unicode
2220
2221
2222
2223
2224
2225
2226
2227
2228
2229
2230
2231
2232
2233
2234
2235
2236
2237
2238
2239
2240
2241
2242
2243
2244
2245
2246
2247
2248
2249
2250
2251
2252
2253
2254
2255
2256
2257
2258
Failed to download XML for PMCPMC9641405. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pm

2827
2828
2829
2830
2831
2832
2833
2834
2835
2836
2837
2838
2839
2840
2841
2842
2843
2844
2845
2846
2847
2848
2849
2850
2851
2852
2853
2854
2855
2856
2857
2858
2859
2860
2861
2862
2863
2864
2865
2866
2867
2868
2869
2870
2871
2872
2873
2874
Failed to download XML for PMCPMC9556450. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC9556450/unicode
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895
2896
2897
2898
2899
2900
2901
2902
2903
2904
2905
2906
2907
2908
2909
2910
2911
2912
2913
2914
2915
2916
2917
2918
2919
2920
2921
2922
2923
2924
2925
2926
2927
2928
2929
2930
2931
2932
2933
2934
2935
2936
2937
2938
2939
2940
2941
2942
2943
2944
2945
2946
2947
2948
2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
Failed to download XML for PMCPMC9443140. HTTP Error: 404 Client Error: Not Found for url: https:/

...

Failed to download XML for PMCPMC1484449. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1484449/unicode
24379
Failed to download XML for PMCPMC1502470. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1502470/unicode
24380
Failed to download XML for PMCPMC2906397. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2906397/unicode
24381
24382
24383
24384
Failed to download XML for PMCPMC1559484. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1559484/unicode
24385
Failed to download XML for PMCPMC2840998. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2840998/unicode
24386
24387
24388
24389
24390
24391
24392
24393


Failed to download XML for PMCPMC2751268. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2751268/unicode
24440
Failed to download XML for PMCPMC1257538. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1257538/unicode
24441
24442
24443
Failed to download XML for PMCPMC2933210. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2933210/unicode
24444
Failed to download XML for PMCPMC2676905. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2676905/unicode
24445
Failed to download XML for PMCPMC1201315. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1201315/unicode
24446
Failed to download XML for PMCPMC1885068. HTTP E

24506
Failed to download XML for PMCPMC539195. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC539195/unicode
24507
24508
Failed to download XML for PMCPMC3004779. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC3004779/unicode
24509
Failed to download XML for PMCPMC2906389. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC2906389/unicode
24510
Failed to download XML for PMCPMC1809264. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1809264/unicode
24511
Failed to download XML for PMCPMC1602292. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1602292/unicode
24512
Failed to download XML for PMCPMC1448861. HTTP Err

Failed to download XML for PMCPMC297010. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC297010/unicode
24561
Couln't parse JSON
24562
Failed to download XML for PMCPMC1773875. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1773875/unicode
24563
Failed to download XML for PMCPMC1223777. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1223777/unicode
24564
Failed to download XML for PMCPMC326652. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC326652/unicode
24565
Failed to download XML for PMCPMC6740907. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC6740907/unicode
24566
24567
Failed to download XML for PMCPMC

Failed to download XML for PMCPMC1531871. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1531871/unicode
24626
Failed to download XML for PMCPMC1508098. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC1508098/unicode
24627
Failed to download XML for PMCPMC4688468. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC4688468/unicode
24628
Failed to download XML for PMCPMC56038. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC56038/unicode
24629
Failed to download XML for PMCPMC93590. HTTP Error: 404 Client Error: Not Found for url: https://www.ncbi.nlm.nih.gov/research/bionlp/RESTful/pmcoa.cgi/BioC_xml/PMC93590/unicode
24630
Failed to download XML for PMCPMC28101. HTTP Error: 404 Client Error

In [99]:
tables_dict = {"PMCID": [], "Tables": []}
for i in range(len(tables_all[0])):
    tables_dict['PMCID'].append(art_list[i])
    tables_dict['Tables'].append(tables_all[0][i])

In [100]:
pd.DataFrame(tables_dict).to_csv("Final_Run_tables.csv")

# Extracting from BIORXIV

In [4]:
import requests
import re
from lxml import etree
from drug_named_entity_recognition import find_drugs

from datetime import datetime, timedelta

def get_papers(start_date, end_date, format='json'):
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    current_year_start = start_date
    papers = []

    print("Starting to fetch papers...")

    with requests.Session() as session:
        while current_year_start <= end_date:
            # End of the current year or the end date, whichever comes first
            current_year_end = datetime(current_year_start.year, 12, 31)
            if current_year_end > end_date:
                current_year_end = end_date
            
            cursor = 0
            total_papers_for_year = None

            while True:
                formatted_start = current_year_start.strftime("%Y-%m-%d")
                formatted_end = current_year_end.strftime("%Y-%m-%d")
                url = f'https://api.biorxiv.org/details/biorxiv/{formatted_start}/{formatted_end}/{cursor}/{format}'
                response = session.get(url)

                if response.status_code == 200:
                    data = response.json()
                    if total_papers_for_year is None:
                        total_papers_for_year = data['messages'][0]['total']
                        print(f"Total papers to fetch for {current_year_start.year}: {total_papers_for_year}")

                    papers_batch = data['collection']
                    papers.extend(papers_batch)
                    print(f"Fetched {len(papers_batch)} papers for {current_year_start.year}; cursor now at {cursor}")

                    cursor += len(papers_batch)  # Update cursor by the number of papers just fetched
                    if len(papers) >= total_papers_for_year or not papers_batch:
                        # All papers for the year have been fetched or no more papers in batch
                        break
                else:
                    print(f"Error fetching data for year {current_year_start.year} at cursor {cursor}: {response.status_code}")
                    break
            
            # Move to the next year
            current_year_start = datetime(current_year_end.year + 1, 1, 1)

    return papers


# Function to extract text from JATS XML
def get_text_from_jats_xml(jats_xml_url):
    response = requests.get(jats_xml_url)
    if response.status_code == 200:
        root = etree.fromstring(response.content)
        text_content = [elem.text for elem in root.xpath('//p') if elem.text]
        return ' '.join(text_content)
    else:
        raise Exception(f'Error fetching XML: {response.status_code}')

# Compile a regular expression for the keywords
bcpr_keywords = ['bcrp', 'abcg2', 'breast cancer resistant protein', 
                 'inhibitor', 'inhibitor', 'transport inhibitor', 'inhibits', 'non transporter', 'unable to transport',
                'non inhibitor', 'substrate', 'transport substrate', 'transporter']

keywords_pattern = re.compile('|'.join(bcpr_keywords), re.IGNORECASE)

# Fetch and filter papers
start_date = '2014-01-01'  # Assuming bioRxiv started in November 2013
end_date = '2023-11-10'
papers = get_papers(start_date, end_date)

# Filter papers based on keywords in title and abstract
filtered_papers = [paper for paper in papers if keywords_pattern.search(paper['title'].lower()) or keywords_pattern.search(paper['abstract'].lower())]


def process_xml(paper):
    try:
        text = get_text_from_jats_xml(paper['jatsxml'])
        
        return paper['doi'], text
    except Exception as e:
        print(f"Error processing paper {paper['doi']}: {e}")
        return paper['doi'], None
    
len(filtered_papers)

Starting to fetch papers...
Total papers to fetch for 2014: 1258
Fetched 100 papers for 2014; cursor now at 0
Fetched 100 papers for 2014; cursor now at 100
Fetched 100 papers for 2014; cursor now at 200
Fetched 100 papers for 2014; cursor now at 300
Fetched 100 papers for 2014; cursor now at 400
Fetched 100 papers for 2014; cursor now at 500
Fetched 100 papers for 2014; cursor now at 600
Fetched 100 papers for 2014; cursor now at 700
Fetched 100 papers for 2014; cursor now at 800
Fetched 100 papers for 2014; cursor now at 900
Fetched 100 papers for 2014; cursor now at 1000
Fetched 100 papers for 2014; cursor now at 1100
Fetched 58 papers for 2014; cursor now at 1200
Total papers to fetch for 2015: 2550
Fetched 100 papers for 2015; cursor now at 0
Fetched 100 papers for 2015; cursor now at 100
Fetched 100 papers for 2015; cursor now at 200
Fetched 100 papers for 2015; cursor now at 300
Fetched 100 papers for 2015; cursor now at 400
Fetched 100 papers for 2015; cursor now at 500
Fetched

Fetched 100 papers for 2018; cursor now at 700
Fetched 100 papers for 2018; cursor now at 800
Fetched 100 papers for 2018; cursor now at 900
Fetched 100 papers for 2018; cursor now at 1000
Fetched 100 papers for 2018; cursor now at 1100
Fetched 100 papers for 2018; cursor now at 1200
Fetched 100 papers for 2018; cursor now at 1300
Fetched 100 papers for 2018; cursor now at 1400
Fetched 100 papers for 2018; cursor now at 1500
Fetched 100 papers for 2018; cursor now at 1600
Fetched 100 papers for 2018; cursor now at 1700
Fetched 100 papers for 2018; cursor now at 1800
Fetched 100 papers for 2018; cursor now at 1900
Fetched 100 papers for 2018; cursor now at 2000
Fetched 100 papers for 2018; cursor now at 2100
Fetched 100 papers for 2018; cursor now at 2200
Fetched 100 papers for 2018; cursor now at 2300
Fetched 100 papers for 2018; cursor now at 2400
Fetched 100 papers for 2018; cursor now at 2500
Fetched 100 papers for 2018; cursor now at 2600
Fetched 100 papers for 2018; cursor now at 

Fetched 100 papers for 2019; cursor now at 5100
Fetched 100 papers for 2019; cursor now at 5200
Fetched 100 papers for 2019; cursor now at 5300
Fetched 100 papers for 2019; cursor now at 5400
Fetched 100 papers for 2019; cursor now at 5500
Fetched 100 papers for 2019; cursor now at 5600
Fetched 100 papers for 2019; cursor now at 5700
Fetched 100 papers for 2019; cursor now at 5800
Fetched 100 papers for 2019; cursor now at 5900
Fetched 100 papers for 2019; cursor now at 6000
Fetched 100 papers for 2019; cursor now at 6100
Fetched 100 papers for 2019; cursor now at 6200
Fetched 100 papers for 2019; cursor now at 6300
Fetched 100 papers for 2019; cursor now at 6400
Fetched 100 papers for 2019; cursor now at 6500
Fetched 100 papers for 2019; cursor now at 6600
Fetched 100 papers for 2019; cursor now at 6700
Fetched 100 papers for 2019; cursor now at 6800
Fetched 100 papers for 2019; cursor now at 6900
Fetched 100 papers for 2019; cursor now at 7000
Fetched 100 papers for 2019; cursor now 

Fetched 100 papers for 2020; cursor now at 10800
Fetched 100 papers for 2020; cursor now at 10900
Fetched 100 papers for 2020; cursor now at 11000
Fetched 100 papers for 2020; cursor now at 11100
Fetched 100 papers for 2020; cursor now at 11200
Fetched 100 papers for 2020; cursor now at 11300
Fetched 100 papers for 2020; cursor now at 11400
Fetched 100 papers for 2020; cursor now at 11500
Fetched 100 papers for 2020; cursor now at 11600
Fetched 100 papers for 2020; cursor now at 11700
Fetched 100 papers for 2020; cursor now at 11800
Fetched 100 papers for 2020; cursor now at 11900
Fetched 100 papers for 2020; cursor now at 12000
Fetched 100 papers for 2020; cursor now at 12100
Fetched 100 papers for 2020; cursor now at 12200
Fetched 100 papers for 2020; cursor now at 12300
Fetched 100 papers for 2020; cursor now at 12400
Fetched 100 papers for 2020; cursor now at 12500
Fetched 100 papers for 2020; cursor now at 12600
Fetched 100 papers for 2020; cursor now at 12700
Fetched 100 papers f

4561

In [5]:
import gc
from drug_named_entity_recognition import find_drugs
import spacy
from spacy.lang.en.stop_words import STOP_WORDS


nlp.max_length = 200000000
def find_entities(filtered_paper):

    doi, text = process_xml(filtered_paper)
    entity = []
    if text != None:
        doc = nlp(text)
        try:
            bcpr_keywords = ["bcrp", "breast cancer resistance protein", "abcg2"]
            sentences = [sent.text for sent in doc.sents]
            for i in range(len(sentences)):
                doc = nlp(sentences[i])
                ner_ = ner(sentences[i])
                has_bcrp = any(token.text.lower() in bcpr_keywords for token in doc)
                if has_bcrp:
                    ents = ner_.ents
                    drugs = []
                    for e in ents:
                        if e.label_ == "CHEMICAL":
                            drugs.append(e.text)
                    for drug in drugs:
                        entity.append({"drug": drug, 'context': doc.text, "Synonyms": None, "DOI": doi})
                    entities = find_drugs([t.text for t in doc], is_ignore_case=True)
                    if len(entities) > 0:
                        for e in entities :
                            if i > 0:
                                previous_sent = sentences[i-1]
                            else:
                                previous_sent = ""
                            if i == len(sentences) - 1:
                                next_sent = ""
                            else:
                                next_sent = sentences[i+1]
                                entity.append({"drug": e[0]['name'], "context": previous_sent + doc.text + next_sent, "Synonyms": e[0]['synonyms'], "DOI": doi})
            if len(entity) > 0:
                return entity
        except:
            pass
    
    
def process_in_batches(filtered_papers, batch_size=100):
    batch_results = []
    x=0
    for index, paper in enumerate(filtered_papers):
        entities = find_entities(paper)
        x+=1
        print(x)
        batch_results.append(entities)
        if (index + 1) % batch_size == 0:
            yield batch_results  # Yield results as a batch
            batch_results.clear()  # Clear the list to free memory
            gc.collect()  # Collect garbage to free memory
    yield batch_results  # Yield any remaining results

batch_size = 200  # Define your batch size

extracted_drugs = []
# x = 0
# for PMCID in pmcid_to_download[0:500]:
#     entities = find_entities(PMCID)
#     extracted_drugs.append(entities)
#     x+= 1
#     if x%50 == 0:
#         print(extracted_drugs)

# Open a file to write the results
with open('BIORXIV_trial.txt', 'w') as output_file:
    for batch in process_in_batches(filtered_papers[1614:], batch_size):
        for entities in batch:
            # Write entities to file or process them
            output_file.write(str(entities) + '\n')
        print(f"Processed a batch of {batch_size}")

1
2
3
4
5
Error processing paper 10.1101/296368: Error fetching XML: 404
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
Error processing paper 10.1101/309476: Error fetching XML: 404
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
Processed a batch of 200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
2

1620
Error processing paper 10.1101/615450: Error fetching XML: 404
1621
1622
1623
1624
1625
1626
1627
1628
1629
1630
1631
1632
1633
1634
1635
1636
1637
1638
1639
1640
1641
1642
1643
1644
1645
1646
1647
1648
1649
1650
1651
1652
1653
1654
1655
1656
1657
1658
1659
1660
1661
1662
1663
1664
1665
1666
1667
1668
1669
1670
1671
1672
1673
1674
1675
1676
1677
1678
1679
1680
1681
1682
1683
1684
1685
1686
1687
1688
1689
1690
1691
1692
1693
1694
1695
1696
1697
1698
1699
1700
1701
1702
1703
1704
1705
1706
1707
1708
Error processing paper 10.1101/721266: Error fetching XML: 404
1709
1710
1711
1712
1713
1714
1715
1716
1717
1718
1719
1720
1721
1722
1723
1724
1725
1726
1727
1728
1729
1730
1731
1732
1733
1734
1735
1736
1737
1738
1739
1740
1741
1742
1743
1744
1745
1746
1747
1748
1749
1750
1751
1752
1753
1754
1755
1756
1757
1758
1759
1760
1761
1762
1763
1764
1765
1766
1767
1768
1769
1770
1771
1772
1773
1774
1775
1776
1777
1778
1779
1780
1781
1782
1783
1784
1785
1786
1787
1788
1789
1790
1791
1792
1793
1794